In [601]:
# /// script
# requires-python = ">=3.11"
# dependencies = [
#     "importlib",
#     "pandas",
#     "udi-grammar-py",``
# ]
# ///

In [602]:
from udi_grammar_py import Chart, Op, rolling
import pandas as pd
import sys
import template_generation
import schema_generation
import template_expansion
import paraphraser
import convert_for_finetuning
import importlib
import json
sys.path.append('.')

## Use python to generate template training data.

In [603]:
importlib.reload(template_generation)
df = template_generation.generate()
template_question_count = df.shape[0]
df.head()

,query_template,constraints,spec_template,query_type,creation_method
0,"How many <E> are there, grouped by <F:n>?","[F.c * 2 < E.c, F.c < 4]","{""source"": {""name"": ""<E>"", ""source"": ""<E.url>""...",question,template


## update data schema based on files in ./datasets folder

In [604]:
importlib.reload(schema_generation)
schema_generation.main()

## Contextualize the template training data by putting in real entity names and fields if they satisfy the constraints.

In [605]:
importlib.reload(template_expansion)
with open('./datasets/schema.json') as f:
    schema_list = json.load(f)
    df = template_expansion.expand(df, schema_list)

df.head()

,query_template,constraints,spec_template,query_type,creation_method,query_base,spec,solution,dataset_schema
0,"How many <E> are there, grouped by <F:n>?","[F.c * 2 < E.c, F.c < 4]","{""source"": {""name"": ""<E>"", ""source"": ""<E.url>""...",question,template,"How many fake_donors are there, grouped by sex?","{""source"": {""name"": ""fake_donors"", ""source"": ""...","{'E': {'entity': 'fake_donors', 'url': './data...",fakePortal
0,"How many <E> are there, grouped by <F:n>?","[F.c * 2 < E.c, F.c < 4]","{""source"": {""name"": ""<E>"", ""source"": ""<E.url>""...",question,template,"How many fake_samples are there, grouped by sa...","{""source"": {""name"": ""fake_samples"", ""source"": ...","{'E': {'entity': 'fake_samples', 'url': './dat...",fakePortal
0,"How many <E> are there, grouped by <F:n>?","[F.c * 2 < E.c, F.c < 4]","{""source"": {""name"": ""<E>"", ""source"": ""<E.url>""...",question,template,"How many fake_samples are there, grouped by or...","{""source"": {""name"": ""fake_samples"", ""source"": ...","{'E': {'entity': 'fake_samples', 'url': './dat...",fakePortal


## The paraphraser will use LLM frameork to paraphrase the query_base into several options


In [606]:
importlib.reload(paraphraser)
df = paraphraser.paraphrase(df)
df.head()


,query_template,constraints,spec_template,query_type,creation_method,query_base,spec,solution,dataset_schema,query,expertise,formality
0,"How many <E> are there, grouped by <F:n>?","[F.c * 2 < E.c, F.c < 4]","{""source"": {""name"": ""<E>"", ""source"": ""<E.url>""...",question,template,"How many fake_donors are there, grouped by sex?","{""source"": {""name"": ""fake_donors"", ""source"": ""...","{'E': {'entity': 'fake_donors', 'url': './data...",fakePortal,"How many fake_donors are there, grouped by sex?",-1,-1
0,"How many <E> are there, grouped by <F:n>?","[F.c * 2 < E.c, F.c < 4]","{""source"": {""name"": ""<E>"", ""source"": ""<E.url>""...",question,template,"How many fake_samples are there, grouped by sa...","{""source"": {""name"": ""fake_samples"", ""source"": ...","{'E': {'entity': 'fake_samples', 'url': './dat...",fakePortal,"How many fake_samples are there, grouped by sa...",-1,-1
0,"How many <E> are there, grouped by <F:n>?","[F.c * 2 < E.c, F.c < 4]","{""source"": {""name"": ""<E>"", ""source"": ""<E.url>""...",question,template,"How many fake_samples are there, grouped by or...","{""source"": {""name"": ""fake_samples"", ""source"": ...","{'E': {'entity': 'fake_samples', 'url': './dat...",fakePortal,"How many fake_samples are there, grouped by or...",-1,-1


## Export as json object

In [607]:
df.to_json('./out/training_data.json', orient='records')

In [608]:
print(df.columns)

Index(['query_template', 'constraints', 'spec_template', 'query_type',
       'creation_method', 'query_base', 'spec', 'solution', 'dataset_schema',
       'query', 'expertise', 'formality'],
      dtype='object')


In [609]:
# print(df.shape)
expanded_question_count = df.shape[0]
print(f"Generated {template_question_count} templates and expanded to {expanded_question_count} questions.")

Generated 1 templates and expanded to 3 questions.


## Convert data frame into format expected for fine tuning

In [ ]:
importlib.reload(convert_for_finetuning)
single_row = df.iloc[[0]]

with open('./datasets/UDIGrammarSchema.json') as grammar_file:
    grammar_schema = json.load(grammar_file)
    convert_for_finetuning.convert(single_row, schema_list, grammar_schema, './out/finetuning_data.json')
